In [1]:
from pathlib import Path
import json
import sys

import gtfstk as gt
import pandas as pd
import numpy as np
import shapely.geometry as sg
import folium

sys.path.append('../')

import gtfs_map_matcher as mm

%load_ext autoreload
%autoreload 2

DATA_DIR = Path('../data')
OUT_DIR = Path('../output')
SECRETS_PATH = Path('../secrets.json')

In [2]:
path = DATA_DIR/'wellington_gtfs_20171016.zip'
feed = gt.read_gtfs(path, dist_units='km')
feed.assess_quality()

,indicator,value
0,num_route_short_names_duplicated,0
1,frac_route_short_names_duplicated,0
2,num_stop_time_dists_missing,0
3,frac_stop_time_dists_missing,0
4,num_direction_ids_missing,0
5,frac_direction_ids_missing,0
6,num_trips_missing_shapes,0
7,frac_trips_missing_shapes,0
8,num_departure_times_missing,0
9,frac_departure_times_missing,0


In [ ]:
# Visually test sample_trip_points()

t = feed.trips.merge(feed.routes)
t = t[t['route_short_name'] == '47'].copy()
%time sample_1 = mm.sample_trip_points(feed, t.trip_id, 5)
%time sample_2 = mm.sample_trip_points(feed, t.trip_id, 100)
%time sample_3 = mm.sample_trip_points(feed, t.trip_id, point_dist=0.1)


In [ ]:
sp = list(sample_1.keys())[0]
l1 = sg.LineString(sample_1[sp])
l2 = sg.LineString(sample_2[sp])
l3 = sg.LineString(sample_3[sp])

# Get actual trip shape
p = mm.get_stop_patterns(feed)
shid = p.loc[p['stop_pattern'] == sp, 'shape_id'].unique()[0]
la = feed.build_geometry_by_shape(shape_ids=[shid], use_utm=False)[shid]

mappy = folium.Map(location=sample_1[sp][0][::-1], zoom_start=14)
folium.GeoJson(sg.mapping(l1), 
  style_function=lambda x: {'color': 'red'}).add_to(mappy)
folium.GeoJson(sg.mapping(l2), 
  style_function=lambda x: {'color': 'orange'}).add_to(mappy)
folium.GeoJson(sg.mapping(l3), 
  style_function=lambda x: {'color': 'yellow'}).add_to(mappy)
folium.GeoJson(sg.mapping(la), 
  style_function=lambda x: {'color': 'green'}).add_to(mappy)
mappy

In [ ]:
# Map match with Mapzen

lm = sg.LineString(mm.match_with_mapzen(sample_2[sp], 
  mm.get_secret(SECRETS_PATH, 'MAPZEN_API_KEY')))

mappy = folium.Map(location=sample_1[sp][0][::-1], zoom_start=14)
folium.GeoJson(sg.mapping(la), 
  style_function=lambda x: {'color': 'green'}).add_to(mappy)
folium.GeoJson(sg.mapping(lm), 
  style_function=lambda x: {'color': 'red'}).add_to(mappy)
mappy


In [ ]:
# Map match with OSRM

lm = sg.LineString(mm.match_with_osrm(sample_2[sp]))

mappy = folium.Map(location=sample_1[sp][0][::-1], zoom_start=14)
folium.GeoJson(sg.mapping(la), 
  style_function=lambda x: {'color': 'green'}).add_to(mappy)
folium.GeoJson(sg.mapping(lm), 
  style_function=lambda x: {'color': 'red'}).add_to(mappy)
mappy


In [ ]:
# Map match with Mapbox, which uses OSRM

lm = sg.LineString(mm.match_with_mapbox(sample_2[sp], 
  mm.get_secret(SECRETS_PATH, 'MAPBOX_API_KEY')))

mappy = folium.Map(location=sample_1[sp][0][::-1], zoom_start=14)
folium.GeoJson(sg.mapping(la), 
  style_function=lambda x: {'color': 'green'}).add_to(mappy)
folium.GeoJson(sg.mapping(lm), 
  style_function=lambda x: {'color': 'red'}).add_to(mappy)
mappy


In [ ]:
# Map match with Google

lm = sg.LineString(mm.match_with_google(sample_2[sp], 
  mm.get_secret(SECRETS_PATH, 'GOOGLE_API_KEY')))

mappy = folium.Map(location=sample_1[sp][0][::-1], zoom_start=14)
folium.GeoJson(sg.mapping(la), 
  style_function=lambda x: {'color': 'green'}).add_to(mappy)
folium.GeoJson(sg.mapping(lm), 
  style_function=lambda x: {'color': 'red'}).add_to(mappy)
mappy


In [5]:
%timeit new_feed = mm.map_match(feed, 'google', mm.get_secret(SECRETS_PATH, 'GOOGLE_API_KEY'))

Map matching 468 stop patterns...
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31


KeyboardInterrupt: 

In [6]:
468*2/60

15.6